In [1]:
import os

In [2]:
%pwd

'd:\\End-to-End-Text-Summerizer_NLP_Project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'd:\\End-to-End-Text-Summerizer_NLP_Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path
@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    data_path: Path
    tokenizer_path: Path
    metric_file_name: Path


In [6]:
from src.EndtoEndTextSummerizer.constants import *
from src.EndtoEndTextSummerizer.utils.common import read_yaml, create_directories 

In [7]:
class ConfigurationManager:
    def __init__(self,config_filepath=CONFIG_FILE_PATH,
                 params_filepath=PARAMS_FILE_PATH):
        
        
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            model_path=config.model_path,
            data_path= config.data_path,
            tokenizer_path= config.tokenizer_path,
            metric_file_name=config.metric_file_name
        )
        return model_evaluation_config

In [8]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm

[2024-01-08 17:46:36,580: INFO: utils: NumExpr defaulting to 8 threads.]


In [9]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config
    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]



    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                                batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                                column_text="article", 
                                column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            # Finally, we decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score
    
    def evaluate(self):
        device = "cuda" if  torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path)

        #loading data
        dataset_samsum_pt = load_from_disk(self.config.data_path)

        rouge_names = ["rouge1","rouge2","rougeL","rougeLsum"]
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
            dataset_samsum_pt['test'][0:10], rouge_metric,model_pegasus,tokenizer,batch_size=2,column_text= 'dialogue', column_summary= 'summary'
        )

        rouge_dict = dict((rn,score[rn].mid.fmeasure) for rn in rouge_names)

        df = pd.DataFrame(rouge_dict, index=['pegasus'])
        df.to_csv(self.config.metric_file_name,index=False)

In [10]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e

[2024-01-08 17:46:40,240: INFO: common: yaml file: config\config.yaml loaded successfully]
[2024-01-08 17:46:40,246: INFO: common: yaml file: params.yaml loaded successfully]
[2024-01-08 17:46:40,250: INFO: common: created directory at: artifacts]
[2024-01-08 17:46:40,253: INFO: common: created directory at: artifacts/model_evaluation]


C:\Users\DELL\AppData\Local\Temp\ipykernel_9228\2576923393.py:55: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')


  0%|          | 0/5 [01:30<?, ?it/s]


KeyboardInterrupt: 